In [1]:
globals().clear
import time
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
from datetime import datetime

In [2]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler

import tensorflow       as tf
import keras.optimizers as op
from keras.callbacks import EarlyStopping

from tensorflow                     import keras
from tensorflow.keras.models        import Sequential
from tensorflow.keras.layers        import Dense
from tensorflow.keras.layers        import Dropout
from tensorflow.keras.optimizers    import Adam

In [3]:
# Load Dataset
df = pd.read_csv('../df.csv')

In [4]:
# list of tickers for stocks in our data set. Sort the ticker list since our df will be alphabetically arranged.
tickers = ["AAPL", 'XOM', 'IBM', 'KO', 'CVX', 'BA', 'PFE', 'MSFT', 'T', 'WMT',
       'F', 'NFLX', 'JPM', 'MCD', 'GE', 'NVDA', 'JNJ', 'BAC', 'C', 'AMZN',
       'INTC', 'CSCO', 'TSLA', 'GOOGL', 'AMD', 'BABA', 'VZ', 'DIS',
       'META', 'PLTR']
tickers.sort()

In [5]:
# For this specific trial drop PLTR since data is not complete
df = df[df.stock_ID != "PLTR"]

In [6]:
df['DATETIME']= pd.to_datetime(df['DATETIME'], format='%m/%d/%Y %H:%M')

In [7]:
# Sort by time so that the first 29 rows occupy the first time value for all the stocks. 
# Fill in 0's for missing values for now.
df = df.set_index('DATETIME')
df = df.fillna(0)

In [8]:
# Select the length of the df ; For this file we need 16 months
# We use the first 12 months to make the first prediction, then shift window
# Then we will repeat this for the next 4 months (Hence total is 16 months needed)
df = df.sort_index().loc['2021-09-01':'2022-12-30']

In [9]:
#Set the DATETIME for fecha 
DATETIME = df.index.values[::29]

In [10]:
# Sort dataframe such that it is both in sequential order, and also in alphabetical order for each day 
#(i.e first entry for each time entry should be AAPL, and last should be XOM).
df.sort_values(["DATETIME", "stock_ID"], inplace=True)

In [11]:
# Select individual stocks and remove demographics
aapl = df.iloc[::29  ,:74]
f    = df.iloc[10::29,:74]
nvda = df.iloc[22::29,:74]
tsla = df.iloc[25::29,:74]
wmt  = df.iloc[27::29,:74]

In [12]:
num_companies = 1  # 29 companies in our dataset.
step_rows  = 24 * num_companies  # 24 time periods per day per stock
total_rows = len(aapl['2021-09-01':'2022-12-30'])  # Define total length to predict on
train_rows = len(aapl['2021-09-01':'2022-08-31'])  # Define length of training window


scale_X = MinMaxScaler()
model_1 = Sequential()
model_1.add(Dense(100, activation = 'relu', input_dim = aapl.iloc[:,1:-1].shape[1]))  # initiating with 100 neurons
model_1.add(Dense(100, activation = 'relu', input_dim = aapl.iloc[:,1:-1].shape[1]))
model_1.add(Dropout(0.25))                                                          # adding dropout to avoid overfitting
model_1.add(Dense(1))                                                               # output layer

opt = Adam(amsgrad = True, lr = 0.001, beta_1 = 0.79, beta_2 = 0.999)             # using Adam optimizer, at a learning rate of 0.001
model_1.compile(loss = 'mse', optimizer = opt)                                      # compiling model
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=4)

aapl_result = pd.DataFrame(columns=['DATETIME', 'ACTUAL', 'PREDICTED', 'DIFFERENCE', 'TRAIN_DURATION'])

for i in range(0, total_rows - train_rows, step_rows):
    st = time.time()
    # 1. Obtain X and y
    train  = aapl.iloc[i:train_rows + i, 1:] 
    test   = aapl.iloc[train_rows + i:train_rows + i + step_rows, 1:]  
    X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
    X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]


    
    # 2. Scale X
    X_train = scale_X.fit_transform(X_train)
    X_test = scale_X.transform(X_test)

    # 3. Fit and Predict
    model_1.fit(X_train, y_train, epochs=200, batch_size=256, callbacks=[es])
    y_hat = model_1.predict(X_test, verbose=False)
    et = time.time()

    # 4. Save data with prediction
    fecha = DATETIME[train_rows + i:train_rows + i + step_rows]
    datos = {
        'DATETIME': fecha.ravel(),
        'ACTUAL': y_test.ravel(),
        'PREDICTED': y_hat.ravel(),
        'DIFFERENCE': abs(y_hat.ravel() - y_test.ravel()),
        'TRAIN_DURATION': np.full(y_hat.ravel().shape[0], et - st)
    }
    data = pd.DataFrame(data=datos)
    aapl_result = pd.concat([aapl_result, data], ignore_index=True)

    print("Count Down:", int((total_rows - train_rows - i) / step_rows))
    # print(data.head())

Epoch 1/200


C:\Users\Gurbir\anaconda3\envs\tf\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


24/24 [==============================] - 2s 4ms/step - loss: 24516.6582
Epoch 2/200
24/24 [==============================] - 0s 4ms/step - loss: 20197.7852
Epoch 3/200
24/24 [==============================] - 0s 4ms/step - loss: 8401.1113
Epoch 4/200
24/24 [==============================] - 0s 4ms/step - loss: 766.2679
Epoch 5/200
24/24 [==============================] - 0s 4ms/step - loss: 325.4783
Epoch 6/200
24/24 [==============================] - 0s 4ms/step - loss: 279.5435
Epoch 7/200
24/24 [==============================] - 0s 4ms/step - loss: 256.2172
Epoch 8/200
24/24 [==============================] - 0s 4ms/step - loss: 237.8343
Epoch 9/200
24/24 [==============================] - 0s 4ms/step - loss: 216.5877
Epoch 10/200
24/24 [==============================] - 0s 4ms/step - loss: 202.8071
Epoch 11/200
24/24 [==============================] - 0s 4ms/step - loss: 202.1146
Epoch 12/200
24/24 [==============================] - 0s 4ms/step - loss: 200.4838
Epoch 13/200
24/24 [

24/24 [==============================] - 0s 4ms/step - loss: 169.9294
Epoch 3/200
24/24 [==============================] - 0s 4ms/step - loss: 166.8367
Epoch 4/200
24/24 [==============================] - 0s 4ms/step - loss: 166.4940
Epoch 5/200
24/24 [==============================] - 0s 4ms/step - loss: 168.9496
Epoch 6/200
24/24 [==============================] - 0s 5ms/step - loss: 167.3416
Epoch 7/200
24/24 [==============================] - 0s 5ms/step - loss: 170.0449
Epoch 8/200
24/24 [==============================] - 0s 4ms/step - loss: 169.0695
Epoch 8: early stopping
Count Down: 60
Epoch 1/200
24/24 [==============================] - 0s 4ms/step - loss: 171.7755
Epoch 2/200
24/24 [==============================] - 0s 4ms/step - loss: 172.6547
Epoch 3/200
24/24 [==============================] - 0s 4ms/step - loss: 170.9233
Epoch 4/200
24/24 [==============================] - 0s 5ms/step - loss: 166.7104
Epoch 5/200
24/24 [==============================] - 0s 5ms/step - loss

24/24 [==============================] - 0s 4ms/step - loss: 169.5162
Epoch 10/200
24/24 [==============================] - 0s 4ms/step - loss: 166.4507
Epoch 11/200
24/24 [==============================] - 0s 4ms/step - loss: 170.9423
Epoch 12/200
24/24 [==============================] - 0s 4ms/step - loss: 172.5604
Epoch 12: early stopping
Count Down: 35
Epoch 1/200
24/24 [==============================] - 0s 4ms/step - loss: 166.9811
Epoch 2/200
24/24 [==============================] - 0s 4ms/step - loss: 167.2416
Epoch 3/200
24/24 [==============================] - 0s 4ms/step - loss: 168.6342
Epoch 4/200
24/24 [==============================] - 0s 4ms/step - loss: 161.8277
Epoch 5/200
24/24 [==============================] - 0s 4ms/step - loss: 166.0592
Epoch 6/200
24/24 [==============================] - 0s 4ms/step - loss: 172.7224
Epoch 7/200
24/24 [==============================] - 0s 4ms/step - loss: 171.1643
Epoch 8/200
24/24 [==============================] - 0s 4ms/step - 

Count Down: 23
Epoch 1/200
24/24 [==============================] - 0s 4ms/step - loss: 166.7972
Epoch 2/200
24/24 [==============================] - 0s 4ms/step - loss: 167.8644
Epoch 3/200
24/24 [==============================] - 0s 4ms/step - loss: 170.4823
Epoch 4/200
24/24 [==============================] - 0s 4ms/step - loss: 166.7072
Epoch 5/200
24/24 [==============================] - 0s 4ms/step - loss: 168.6033
Epoch 6/200
24/24 [==============================] - 0s 4ms/step - loss: 164.9874
Epoch 7/200
24/24 [==============================] - 0s 4ms/step - loss: 161.9877
Epoch 8/200
24/24 [==============================] - 0s 4ms/step - loss: 167.3862
Epoch 9/200
24/24 [==============================] - 0s 5ms/step - loss: 165.6536
Epoch 10/200
24/24 [==============================] - 0s 4ms/step - loss: 166.1321
Epoch 11/200
24/24 [==============================] - 0s 5ms/step - loss: 166.9066
Epoch 11: early stopping
Count Down: 22
Epoch 1/200
24/24 [======================

In [13]:
num_companies = 1  # 29 companies in our dataset.
step_rows  = 24 * num_companies  # 24 time periods per day per stock
total_rows = len(f['2021-09-01':'2022-12-30'])  # Define total length to predict on
train_rows = len(f['2021-09-01':'2022-08-31'])  # Define length of training window


scale_X = MinMaxScaler()
model_2 = Sequential()
model_2.add(Dense(100, activation = 'relu', input_dim = f.iloc[:,1:-1].shape[1]))  # initiating with 100 neurons
model_2.add(Dense(100, activation = 'relu', input_dim = f.iloc[:,1:-1].shape[1]))
model_2.add(Dropout(0.25))                                                          # adding dropout to avoid overfitting
model_2.add(Dense(1))                                                               # output layer

opt = Adam(amsgrad = True, lr = 0.001, beta_1 = 0.79, beta_2 = 0.999)             # using Adam optimizer, at a learning rate of 0.001
model_2.compile(loss = 'mse', optimizer = opt)                                      # compiling model
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=4)

f_result = pd.DataFrame(columns=['DATETIME', 'ACTUAL', 'PREDICTED', 'DIFFERENCE', 'TRAIN_DURATION'])

for i in range(0, total_rows - train_rows, step_rows):
    st = time.time()
    # 1. Obtain X and y
    train  = f.iloc[i:train_rows + i, 1:] 
    test   = f.iloc[train_rows + i:train_rows + i + step_rows, 1:]  
    X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
    X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]


    
    # 2. Scale X
    X_train = scale_X.fit_transform(X_train)
    X_test = scale_X.transform(X_test)

    # 3. Fit and Predict
    model_2.fit(X_train, y_train, epochs=200, batch_size=256, callbacks=[es])
    y_hat = model_2.predict(X_test, verbose=False)
    et = time.time()

    # 4. Save data with prediction
    fecha = DATETIME[train_rows + i:train_rows + i + step_rows]
    datos = {
        'DATETIME': fecha.ravel(),
        'ACTUAL': y_test.ravel(),
        'PREDICTED': y_hat.ravel(),
        'DIFFERENCE': abs(y_hat.ravel() - y_test.ravel()),
        'TRAIN_DURATION': np.full(y_hat.ravel().shape[0], et - st)
    }
    data = pd.DataFrame(data=datos)
    f_result = pd.concat([f_result, data], ignore_index=True)

    print("Count Down:", int((total_rows - train_rows - i) / step_rows))
    # print(data.head())

Epoch 1/200


C:\Users\Gurbir\anaconda3\envs\tf\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


24/24 [==============================] - 0s 4ms/step - loss: 217.1155
Epoch 2/200
24/24 [==============================] - 0s 4ms/step - loss: 47.1197
Epoch 3/200
24/24 [==============================] - 0s 4ms/step - loss: 10.4640
Epoch 4/200
24/24 [==============================] - 0s 5ms/step - loss: 6.4092
Epoch 5/200
24/24 [==============================] - 0s 4ms/step - loss: 4.5502
Epoch 6/200
24/24 [==============================] - 0s 4ms/step - loss: 3.5462
Epoch 7/200
24/24 [==============================] - 0s 4ms/step - loss: 3.2046
Epoch 8/200
24/24 [==============================] - 0s 4ms/step - loss: 2.9828
Epoch 9/200
24/24 [==============================] - 0s 4ms/step - loss: 2.7758
Epoch 10/200
24/24 [==============================] - 0s 4ms/step - loss: 2.7642
Epoch 11/200
24/24 [==============================] - 0s 4ms/step - loss: 2.7368
Epoch 12/200
24/24 [==============================] - 0s 5ms/step - loss: 2.6553
Epoch 13/200
24/24 [=========================

24/24 [==============================] - 0s 4ms/step - loss: 2.0652
Epoch 3/200
24/24 [==============================] - 0s 4ms/step - loss: 2.0724
Epoch 4/200
24/24 [==============================] - 0s 4ms/step - loss: 2.0824
Epoch 5/200
24/24 [==============================] - 0s 4ms/step - loss: 2.0354
Epoch 6/200
24/24 [==============================] - 0s 4ms/step - loss: 1.9868
Epoch 7/200
24/24 [==============================] - 0s 4ms/step - loss: 2.0369
Epoch 8/200
24/24 [==============================] - 0s 4ms/step - loss: 2.0961
Epoch 9/200
24/24 [==============================] - 0s 4ms/step - loss: 2.0246
Epoch 10/200
24/24 [==============================] - 0s 4ms/step - loss: 1.9638
Epoch 11/200
24/24 [==============================] - 0s 4ms/step - loss: 2.0275
Epoch 12/200
24/24 [==============================] - 0s 4ms/step - loss: 2.0359
Epoch 13/200
24/24 [==============================] - 0s 4ms/step - loss: 1.9622
Epoch 14/200
24/24 [============================

Epoch 4/200
24/24 [==============================] - 0s 5ms/step - loss: 1.9128
Epoch 5/200
24/24 [==============================] - 0s 5ms/step - loss: 1.8189
Epoch 5: early stopping
Count Down: 40
Epoch 1/200
24/24 [==============================] - 0s 5ms/step - loss: 1.9243
Epoch 2/200
24/24 [==============================] - 0s 5ms/step - loss: 1.9552
Epoch 3/200
24/24 [==============================] - 0s 5ms/step - loss: 1.8882
Epoch 4/200
24/24 [==============================] - 0s 5ms/step - loss: 1.9101
Epoch 5/200
24/24 [==============================] - 0s 6ms/step - loss: 1.8970
Epoch 6/200
24/24 [==============================] - 0s 5ms/step - loss: 1.9319
Epoch 7/200
24/24 [==============================] - 0s 5ms/step - loss: 1.8953
Epoch 7: early stopping
Count Down: 39
Epoch 1/200
24/24 [==============================] - 0s 6ms/step - loss: 1.8798
Epoch 2/200
24/24 [==============================] - 0s 6ms/step - loss: 1.9446
Epoch 3/200
24/24 [=======================

Epoch 1/200
24/24 [==============================] - 0s 5ms/step - loss: 1.7961
Epoch 2/200
24/24 [==============================] - 0s 5ms/step - loss: 1.8062
Epoch 3/200
24/24 [==============================] - 0s 5ms/step - loss: 1.7465
Epoch 4/200
24/24 [==============================] - 0s 5ms/step - loss: 1.7910
Epoch 5/200
24/24 [==============================] - 0s 5ms/step - loss: 1.7998
Epoch 6/200
24/24 [==============================] - 0s 5ms/step - loss: 1.7745
Epoch 7/200
24/24 [==============================] - 0s 4ms/step - loss: 1.8426
Epoch 7: early stopping
Count Down: 27
Epoch 1/200
24/24 [==============================] - 0s 4ms/step - loss: 1.7378
Epoch 2/200
24/24 [==============================] - 0s 4ms/step - loss: 1.7600
Epoch 3/200
24/24 [==============================] - 0s 4ms/step - loss: 1.7256
Epoch 4/200
24/24 [==============================] - 0s 4ms/step - loss: 1.7250
Epoch 5/200
24/24 [==============================] - 0s 4ms/step - loss: 1.8065
E

24/24 [==============================] - 0s 4ms/step - loss: 1.5959
Epoch 4/200
24/24 [==============================] - 0s 4ms/step - loss: 1.6269
Epoch 5/200
24/24 [==============================] - 0s 5ms/step - loss: 1.6578
Epoch 6/200
24/24 [==============================] - 0s 4ms/step - loss: 1.5757
Epoch 7/200
24/24 [==============================] - 0s 4ms/step - loss: 1.5454
Epoch 8/200
24/24 [==============================] - 0s 4ms/step - loss: 1.6360
Epoch 9/200
24/24 [==============================] - 0s 4ms/step - loss: 1.5952
Epoch 10/200
24/24 [==============================] - 0s 4ms/step - loss: 1.6667
Epoch 11/200
24/24 [==============================] - 0s 4ms/step - loss: 1.5701
Epoch 11: early stopping
Count Down: 6
Epoch 1/200
24/24 [==============================] - 0s 5ms/step - loss: 1.6537
Epoch 2/200
24/24 [==============================] - 0s 4ms/step - loss: 1.6014
Epoch 3/200
24/24 [==============================] - 0s 5ms/step - loss: 1.6113
Epoch 4/200

In [14]:
num_companies = 1  # 29 companies in our dataset.
step_rows  = 24 * num_companies  # 24 time periods per day per stock
total_rows = len(nvda['2021-09-01':'2022-12-30'])  # Define total length to predict on
train_rows = len(nvda['2021-09-01':'2022-08-31'])  # Define length of training window


scale_X = MinMaxScaler()
model_3 = Sequential()
model_3.add(Dense(100, activation = 'relu', input_dim = nvda.iloc[:,1:-1].shape[1]))  # initiating with 100 neurons
model_3.add(Dense(100, activation = 'relu', input_dim = nvda.iloc[:,1:-1].shape[1]))
model_3.add(Dropout(0.25))                                                          # adding dropout to avoid overfitting
model_3.add(Dense(1))                                                               # output layer

opt = Adam(amsgrad = True, lr = 0.001, beta_1 = 0.79, beta_2 = 0.999)             # using Adam optimizer, at a learning rate of 0.001
model_3.compile(loss = 'mse', optimizer = opt)                                      # compiling model
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=4)

nvda_result = pd.DataFrame(columns=['DATETIME', 'ACTUAL', 'PREDICTED', 'DIFFERENCE', 'TRAIN_DURATION'])

for i in range(0, total_rows - train_rows, step_rows):
    st = time.time()
    # 1. Obtain X and y
    train  = nvda.iloc[i:train_rows + i, 1:] 
    test   = nvda.iloc[train_rows + i:train_rows + i + step_rows, 1:]  
    X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
    X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]


    
    # 2. Scale X
    X_train = scale_X.fit_transform(X_train)
    X_test = scale_X.transform(X_test)

    # 3. Fit and Predict
    model_3.fit(X_train, y_train, epochs=200, batch_size=256, callbacks=[es])
    y_hat = model_3.predict(X_test, verbose=False)
    et = time.time()

    # 4. Save data with prediction
    fecha = DATETIME[train_rows + i:train_rows + i + step_rows]
    datos = {
        'DATETIME': fecha.ravel(),
        'ACTUAL': y_test.ravel(),
        'PREDICTED': y_hat.ravel(),
        'DIFFERENCE': abs(y_hat.ravel() - y_test.ravel()),
        'TRAIN_DURATION': np.full(y_hat.ravel().shape[0], et - st)
    }
    data = pd.DataFrame(data=datos)
    nvda_result = pd.concat([nvda_result, data], ignore_index=True)

    print("Count Down:", int((total_rows - train_rows - i) / step_rows))
    # print(data.head())

Epoch 1/200


C:\Users\Gurbir\anaconda3\envs\tf\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


24/24 [==============================] - 0s 5ms/step - loss: 52067.2734
Epoch 2/200
24/24 [==============================] - 0s 4ms/step - loss: 46546.4336
Epoch 3/200
24/24 [==============================] - 0s 5ms/step - loss: 29308.7852
Epoch 4/200
24/24 [==============================] - 0s 4ms/step - loss: 8527.7598
Epoch 5/200
24/24 [==============================] - 0s 4ms/step - loss: 2616.7295
Epoch 6/200
24/24 [==============================] - 0s 5ms/step - loss: 1884.6980
Epoch 7/200
24/24 [==============================] - 0s 5ms/step - loss: 1431.8068
Epoch 8/200
24/24 [==============================] - 0s 5ms/step - loss: 1146.9717
Epoch 9/200
24/24 [==============================] - 0s 5ms/step - loss: 908.0074
Epoch 10/200
24/24 [==============================] - 0s 5ms/step - loss: 776.9600
Epoch 11/200
24/24 [==============================] - 0s 4ms/step - loss: 719.6414
Epoch 12/200
24/24 [==============================] - 0s 5ms/step - loss: 633.6880
Epoch 13/200
2

24/24 [==============================] - 0s 4ms/step - loss: 436.9051
Epoch 6/200
24/24 [==============================] - 0s 4ms/step - loss: 429.3426
Epoch 6: early stopping
Count Down: 74
Epoch 1/200
24/24 [==============================] - 0s 5ms/step - loss: 443.9094
Epoch 2/200
24/24 [==============================] - 0s 4ms/step - loss: 432.1441
Epoch 3/200
24/24 [==============================] - 0s 4ms/step - loss: 440.8254
Epoch 4/200
24/24 [==============================] - 0s 4ms/step - loss: 437.5022
Epoch 5/200
24/24 [==============================] - 0s 4ms/step - loss: 447.0205
Epoch 6/200
24/24 [==============================] - 0s 4ms/step - loss: 441.1658
Epoch 6: early stopping
Count Down: 73
Epoch 1/200
24/24 [==============================] - 0s 4ms/step - loss: 443.7558
Epoch 2/200
24/24 [==============================] - 0s 4ms/step - loss: 424.6199
Epoch 3/200
24/24 [==============================] - 0s 4ms/step - loss: 430.6198
Epoch 4/200
24/24 [=============

24/24 [==============================] - 0s 5ms/step - loss: 368.8987
Epoch 12/200
24/24 [==============================] - 0s 5ms/step - loss: 380.8794
Epoch 12: early stopping
Count Down: 54
Epoch 1/200
24/24 [==============================] - 0s 5ms/step - loss: 376.1446
Epoch 2/200
24/24 [==============================] - 0s 5ms/step - loss: 377.3224
Epoch 3/200
24/24 [==============================] - 0s 5ms/step - loss: 365.0521
Epoch 4/200
24/24 [==============================] - 0s 5ms/step - loss: 379.3072
Epoch 5/200
24/24 [==============================] - 0s 5ms/step - loss: 361.0893
Epoch 6/200
24/24 [==============================] - 0s 5ms/step - loss: 374.6819
Epoch 7/200
24/24 [==============================] - 0s 5ms/step - loss: 369.1422
Epoch 8/200
24/24 [==============================] - 0s 4ms/step - loss: 376.3261
Epoch 9/200
24/24 [==============================] - 0s 5ms/step - loss: 380.1384
Epoch 9: early stopping
Count Down: 53
Epoch 1/200
24/24 [===========

24/24 [==============================] - 0s 4ms/step - loss: 321.3136
Epoch 8/200
24/24 [==============================] - 0s 4ms/step - loss: 318.7054
Epoch 9/200
24/24 [==============================] - 0s 5ms/step - loss: 321.1339
Epoch 10/200
24/24 [==============================] - 0s 6ms/step - loss: 326.3857
Epoch 11/200
24/24 [==============================] - 0s 5ms/step - loss: 319.3690
Epoch 12/200
24/24 [==============================] - 0s 5ms/step - loss: 313.9609
Epoch 13/200
24/24 [==============================] - 0s 5ms/step - loss: 314.3723
Epoch 14/200
24/24 [==============================] - 0s 6ms/step - loss: 318.0056
Epoch 15/200
24/24 [==============================] - 0s 5ms/step - loss: 321.8810
Epoch 16/200
24/24 [==============================] - 0s 5ms/step - loss: 326.9451
Epoch 16: early stopping
Count Down: 32
Epoch 1/200
24/24 [==============================] - 0s 5ms/step - loss: 317.5497
Epoch 2/200
24/24 [==============================] - 0s 6ms/ste

24/24 [==============================] - 0s 5ms/step - loss: 287.5654
Epoch 7/200
24/24 [==============================] - 0s 4ms/step - loss: 269.2838
Epoch 8/200
24/24 [==============================] - 0s 4ms/step - loss: 276.2427
Epoch 9/200
24/24 [==============================] - 0s 4ms/step - loss: 287.1662
Epoch 10/200
24/24 [==============================] - 0s 4ms/step - loss: 272.8719
Epoch 11/200
24/24 [==============================] - 0s 4ms/step - loss: 272.4702
Epoch 11: early stopping
Count Down: 11
Epoch 1/200
24/24 [==============================] - 0s 4ms/step - loss: 276.0555
Epoch 2/200
24/24 [==============================] - 0s 4ms/step - loss: 273.3694
Epoch 3/200
24/24 [==============================] - 0s 4ms/step - loss: 276.2443
Epoch 4/200
24/24 [==============================] - 0s 4ms/step - loss: 267.9757
Epoch 5/200
24/24 [==============================] - 0s 4ms/step - loss: 264.8864
Epoch 6/200
24/24 [==============================] - 0s 4ms/step - l

In [15]:
num_companies = 1  # 29 companies in our dataset.
step_rows  = 24 * num_companies  # 24 time periods per day per stock
total_rows = len(tsla['2021-09-01':'2022-12-30'])  # Define total length to predict on
train_rows = len(tsla['2021-09-01':'2022-08-31'])  # Define length of training window


scale_X = MinMaxScaler()
model_4 = Sequential()
model_4.add(Dense(100, activation = 'relu', input_dim = tsla.iloc[:,1:-1].shape[1]))  # initiating with 100 neurons
model_4.add(Dense(100, activation = 'relu', input_dim = tsla.iloc[:,1:-1].shape[1]))
model_4.add(Dropout(0.25))                                                          # adding dropout to avoid overfitting
model_4.add(Dense(1))                                                               # output layer

opt = Adam(amsgrad = True, lr = 0.001, beta_1 = 0.79, beta_2 = 0.999)             # using Adam optimizer, at a learning rate of 0.001
model_4.compile(loss = 'mse', optimizer = opt)                                      # compiling model
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=4)

tsla_result = pd.DataFrame(columns=['DATETIME', 'ACTUAL', 'PREDICTED', 'DIFFERENCE', 'TRAIN_DURATION'])

for i in range(0, total_rows - train_rows, step_rows):
    st = time.time()
    # 1. Obtain X and y
    train  = tsla.iloc[i:train_rows + i, 1:] 
    test   = tsla.iloc[train_rows + i:train_rows + i + step_rows, 1:]  
    X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
    X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]


    
    # 2. Scale X
    X_train = scale_X.fit_transform(X_train)
    X_test = scale_X.transform(X_test)

    # 3. Fit and Predict
    model_4.fit(X_train, y_train, epochs=200, batch_size=256, callbacks=[es])
    y_hat = model_4.predict(X_test, verbose=False)
    et = time.time()

    # 4. Save data with prediction
    fecha = DATETIME[train_rows + i:train_rows + i + step_rows]
    datos = {
        'DATETIME': fecha.ravel(),
        'ACTUAL': y_test.ravel(),
        'PREDICTED': y_hat.ravel(),
        'DIFFERENCE': abs(y_hat.ravel() - y_test.ravel()),
        'TRAIN_DURATION': np.full(y_hat.ravel().shape[0], et - st)
    }
    data = pd.DataFrame(data=datos)
    tsla_result = pd.concat([tsla_result, data], ignore_index=True)

    print("Count Down:", int((total_rows - train_rows - i) / step_rows))
    # print(data.head())

Epoch 1/200


C:\Users\Gurbir\anaconda3\envs\tf\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


24/24 [==============================] - 0s 4ms/step - loss: 88897.7500
Epoch 2/200
24/24 [==============================] - 0s 4ms/step - loss: 80706.9141
Epoch 3/200
24/24 [==============================] - 0s 4ms/step - loss: 52078.0273
Epoch 4/200
24/24 [==============================] - 0s 4ms/step - loss: 15141.6738
Epoch 5/200
24/24 [==============================] - 0s 5ms/step - loss: 2852.3953
Epoch 6/200
24/24 [==============================] - 0s 5ms/step - loss: 2025.8569
Epoch 7/200
24/24 [==============================] - 0s 4ms/step - loss: 1781.4843
Epoch 8/200
24/24 [==============================] - 0s 4ms/step - loss: 1486.6403
Epoch 9/200
24/24 [==============================] - 0s 4ms/step - loss: 1280.5403
Epoch 10/200
24/24 [==============================] - 0s 5ms/step - loss: 1178.9911
Epoch 11/200
24/24 [==============================] - 0s 4ms/step - loss: 1081.7897
Epoch 12/200
24/24 [==============================] - 0s 4ms/step - loss: 1015.6375
Epoch 13/

24/24 [==============================] - 0s 4ms/step - loss: 680.9011
Epoch 2/200
24/24 [==============================] - 0s 4ms/step - loss: 695.5156
Epoch 3/200
24/24 [==============================] - 0s 4ms/step - loss: 701.7978
Epoch 4/200
24/24 [==============================] - 0s 4ms/step - loss: 692.7516
Epoch 5/200
24/24 [==============================] - 0s 4ms/step - loss: 720.7433
Epoch 5: early stopping
Count Down: 61
Epoch 1/200
24/24 [==============================] - 0s 4ms/step - loss: 646.7910
Epoch 2/200
24/24 [==============================] - 0s 6ms/step - loss: 677.0507
Epoch 3/200
24/24 [==============================] - 0s 5ms/step - loss: 714.8680
Epoch 4/200
24/24 [==============================] - 0s 5ms/step - loss: 692.0598
Epoch 5/200
24/24 [==============================] - 0s 4ms/step - loss: 676.5093
Epoch 5: early stopping
Count Down: 60
Epoch 1/200
24/24 [==============================] - 0s 5ms/step - loss: 681.7770
Epoch 2/200
24/24 [=============

24/24 [==============================] - 0s 5ms/step - loss: 656.5146
Epoch 7/200
24/24 [==============================] - 0s 4ms/step - loss: 639.6304
Epoch 8/200
24/24 [==============================] - 0s 5ms/step - loss: 673.0337
Epoch 9/200
24/24 [==============================] - 0s 4ms/step - loss: 659.5346
Epoch 10/200
24/24 [==============================] - 0s 5ms/step - loss: 652.4741
Epoch 11/200
24/24 [==============================] - 0s 5ms/step - loss: 674.5292
Epoch 11: early stopping
Count Down: 51
Epoch 1/200
24/24 [==============================] - 0s 5ms/step - loss: 651.3305
Epoch 2/200
24/24 [==============================] - 0s 5ms/step - loss: 669.7237
Epoch 3/200
24/24 [==============================] - 0s 5ms/step - loss: 655.8859
Epoch 4/200
24/24 [==============================] - 0s 5ms/step - loss: 657.5366
Epoch 5/200
24/24 [==============================] - 0s 5ms/step - loss: 671.6686
Epoch 5: early stopping
Count Down: 50
Epoch 1/200
24/24 [==========

Epoch 5/200
24/24 [==============================] - 0s 4ms/step - loss: 640.5536
Epoch 6/200
24/24 [==============================] - 0s 5ms/step - loss: 652.8596
Epoch 7/200
24/24 [==============================] - 0s 5ms/step - loss: 639.7060
Epoch 8/200
24/24 [==============================] - 0s 5ms/step - loss: 667.5387
Epoch 9/200
24/24 [==============================] - 0s 5ms/step - loss: 637.8331
Epoch 10/200
24/24 [==============================] - 0s 5ms/step - loss: 662.6291
Epoch 11/200
24/24 [==============================] - 0s 5ms/step - loss: 655.5208
Epoch 12/200
24/24 [==============================] - 0s 5ms/step - loss: 653.4326
Epoch 13/200
24/24 [==============================] - 0s 5ms/step - loss: 653.5508
Epoch 13: early stopping
Count Down: 41
Epoch 1/200
24/24 [==============================] - 0s 5ms/step - loss: 667.3887
Epoch 2/200
24/24 [==============================] - 0s 5ms/step - loss: 649.9629
Epoch 3/200
24/24 [==============================] - 0

24/24 [==============================] - 0s 5ms/step - loss: 573.3253
Epoch 6/200
24/24 [==============================] - 0s 5ms/step - loss: 574.0531
Epoch 7/200
24/24 [==============================] - 0s 5ms/step - loss: 575.0256
Epoch 8/200
24/24 [==============================] - 0s 5ms/step - loss: 577.0483
Epoch 9/200
24/24 [==============================] - 0s 5ms/step - loss: 568.5671
Epoch 10/200
24/24 [==============================] - 0s 5ms/step - loss: 582.4136
Epoch 11/200
24/24 [==============================] - 0s 5ms/step - loss: 553.4138
Epoch 12/200
24/24 [==============================] - 0s 5ms/step - loss: 574.4269
Epoch 13/200
24/24 [==============================] - 0s 5ms/step - loss: 579.0338
Epoch 14/200
24/24 [==============================] - 0s 5ms/step - loss: 559.4879
Epoch 15/200
24/24 [==============================] - 0s 5ms/step - loss: 575.0549
Epoch 15: early stopping
Count Down: 21
Epoch 1/200
24/24 [==============================] - 0s 5ms/step

24/24 [==============================] - 0s 5ms/step - loss: 487.6092
Epoch 2/200
24/24 [==============================] - 0s 5ms/step - loss: 484.1678
Epoch 3/200
24/24 [==============================] - 0s 5ms/step - loss: 491.2341
Epoch 4/200
24/24 [==============================] - 0s 5ms/step - loss: 494.3892
Epoch 5/200
24/24 [==============================] - 0s 5ms/step - loss: 485.9319
Epoch 6/200
24/24 [==============================] - 0s 5ms/step - loss: 482.4555
Epoch 7/200
24/24 [==============================] - 0s 5ms/step - loss: 487.7818
Epoch 8/200
24/24 [==============================] - 0s 5ms/step - loss: 476.2453
Epoch 9/200
24/24 [==============================] - 0s 5ms/step - loss: 491.3492
Epoch 10/200
24/24 [==============================] - 0s 5ms/step - loss: 480.2440
Epoch 11/200
24/24 [==============================] - 0s 5ms/step - loss: 486.7833
Epoch 12/200
24/24 [==============================] - 0s 5ms/step - loss: 480.1942
Epoch 12: early stopping


In [16]:
num_companies = 1  # 29 companies in our dataset.
step_rows  = 24 * num_companies  # 24 time periods per day per stock
total_rows = len(wmt['2021-09-01':'2022-12-30'])  # Define total length to predict on
train_rows = len(wmt['2021-09-01':'2022-08-31'])  # Define length of training window


scale_X = MinMaxScaler()
model_5 = Sequential()
model_5.add(Dense(100, activation = 'relu', input_dim = wmt.iloc[:,1:-1].shape[1]))  # initiating with 100 neurons
model_5.add(Dense(100, activation = 'relu', input_dim = wmt.iloc[:,1:-1].shape[1]))
model_5.add(Dropout(0.25))                                                          # adding dropout to avoid overfitting
model_5.add(Dense(1))                                                               # output layer

opt = Adam(amsgrad = True, lr = 0.001, beta_1 = 0.79, beta_2 = 0.999)             # using Adam optimizer, at a learning rate of 0.001
model_5.compile(loss = 'mse', optimizer = opt)                                      # compiling model
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=4)

wmt_result = pd.DataFrame(columns=['DATETIME', 'ACTUAL', 'PREDICTED', 'DIFFERENCE', 'TRAIN_DURATION'])

for i in range(0, total_rows - train_rows, step_rows):
    st = time.time()
    # 1. Obtain X and y
    train  = wmt.iloc[i:train_rows + i, 1:] 
    test   = wmt.iloc[train_rows + i:train_rows + i + step_rows, 1:]  
    X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
    X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]


    
    # 2. Scale X
    X_train = scale_X.fit_transform(X_train)
    X_test = scale_X.transform(X_test)

    # 3. Fit and Predict
    model_5.fit(X_train, y_train, epochs=200, batch_size=256, callbacks=[es])
    y_hat = model_5.predict(X_test, verbose=False)
    et = time.time()

    # 4. Save data with prediction
    fecha = DATETIME[train_rows + i:train_rows + i + step_rows]
    datos = {
        'DATETIME': fecha.ravel(),
        'ACTUAL': y_test.ravel(),
        'PREDICTED': y_hat.ravel(),
        'DIFFERENCE': abs(y_hat.ravel() - y_test.ravel()),
        'TRAIN_DURATION': np.full(y_hat.ravel().shape[0], et - st)
    }
    data = pd.DataFrame(data=datos)
    wmt_result = pd.concat([wmt_result, data], ignore_index=True)

    print("Count Down:", int((total_rows - train_rows - i) / step_rows))
    # print(data.head())

Epoch 1/200


C:\Users\Gurbir\anaconda3\envs\tf\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


24/24 [==============================] - 1s 5ms/step - loss: 19129.6367
Epoch 2/200
24/24 [==============================] - 0s 5ms/step - loss: 16112.8076
Epoch 3/200
24/24 [==============================] - 0s 5ms/step - loss: 7160.6177
Epoch 4/200
24/24 [==============================] - 0s 4ms/step - loss: 718.0485
Epoch 5/200
24/24 [==============================] - 0s 5ms/step - loss: 277.2197
Epoch 6/200
24/24 [==============================] - 0s 5ms/step - loss: 242.0992
Epoch 7/200
24/24 [==============================] - 0s 5ms/step - loss: 217.9106
Epoch 8/200
24/24 [==============================] - 0s 5ms/step - loss: 203.7332
Epoch 9/200
24/24 [==============================] - 0s 5ms/step - loss: 198.9542
Epoch 10/200
24/24 [==============================] - 0s 5ms/step - loss: 188.0535
Epoch 11/200
24/24 [==============================] - 0s 6ms/step - loss: 183.4311
Epoch 12/200
24/24 [==============================] - 0s 5ms/step - loss: 175.1290
Epoch 13/200
24/24 [

24/24 [==============================] - 0s 5ms/step - loss: 146.4349
Epoch 5/200
24/24 [==============================] - 0s 5ms/step - loss: 142.8186
Epoch 6/200
24/24 [==============================] - 0s 5ms/step - loss: 140.3700
Epoch 7/200
24/24 [==============================] - 0s 5ms/step - loss: 144.4128
Epoch 8/200
24/24 [==============================] - 0s 5ms/step - loss: 140.1409
Epoch 9/200
24/24 [==============================] - 0s 5ms/step - loss: 140.7881
Epoch 10/200
24/24 [==============================] - 0s 5ms/step - loss: 141.1501
Epoch 11/200
24/24 [==============================] - 0s 5ms/step - loss: 139.4983
Epoch 12/200
24/24 [==============================] - 0s 5ms/step - loss: 141.4323
Epoch 13/200
24/24 [==============================] - 0s 5ms/step - loss: 140.7128
Epoch 14/200
24/24 [==============================] - 0s 5ms/step - loss: 142.1561
Epoch 15/200
24/24 [==============================] - 0s 5ms/step - loss: 143.1391
Epoch 15: early stoppi

24/24 [==============================] - 0s 5ms/step - loss: 142.1286
Epoch 7/200
24/24 [==============================] - 0s 5ms/step - loss: 140.1805
Epoch 8/200
24/24 [==============================] - 0s 5ms/step - loss: 137.5332
Epoch 9/200
24/24 [==============================] - 0s 5ms/step - loss: 139.2410
Epoch 10/200
24/24 [==============================] - 0s 5ms/step - loss: 133.8210
Epoch 11/200
24/24 [==============================] - 0s 5ms/step - loss: 140.8183
Epoch 12/200
24/24 [==============================] - 0s 5ms/step - loss: 140.7999
Epoch 13/200
24/24 [==============================] - 0s 5ms/step - loss: 139.7499
Epoch 14/200
24/24 [==============================] - 0s 5ms/step - loss: 137.4740
Epoch 14: early stopping
Count Down: 52
Epoch 1/200
24/24 [==============================] - 0s 5ms/step - loss: 137.8208
Epoch 2/200
24/24 [==============================] - 0s 5ms/step - loss: 142.1311
Epoch 3/200
24/24 [==============================] - 0s 5ms/step 

24/24 [==============================] - 0s 5ms/step - loss: 135.6111
Epoch 6/200
24/24 [==============================] - 0s 5ms/step - loss: 141.4525
Epoch 7/200
24/24 [==============================] - 0s 5ms/step - loss: 134.7301
Epoch 8/200
24/24 [==============================] - 0s 5ms/step - loss: 132.7561
Epoch 9/200
24/24 [==============================] - 0s 5ms/step - loss: 139.1296
Epoch 10/200
24/24 [==============================] - 0s 5ms/step - loss: 138.2254
Epoch 11/200
24/24 [==============================] - 0s 5ms/step - loss: 137.2075
Epoch 12/200
24/24 [==============================] - 0s 5ms/step - loss: 138.1241
Epoch 12: early stopping
Count Down: 31
Epoch 1/200
24/24 [==============================] - 0s 5ms/step - loss: 138.1301
Epoch 2/200
24/24 [==============================] - 0s 5ms/step - loss: 135.6287
Epoch 3/200
24/24 [==============================] - 0s 5ms/step - loss: 133.4281
Epoch 4/200
24/24 [==============================] - 0s 6ms/step - 

24/24 [==============================] - 0s 4ms/step - loss: 138.6963
Epoch 7/200
24/24 [==============================] - 0s 4ms/step - loss: 139.3901
Epoch 7: early stopping
Count Down: 8
Epoch 1/200
24/24 [==============================] - 0s 4ms/step - loss: 140.8563
Epoch 2/200
24/24 [==============================] - 0s 4ms/step - loss: 137.5929
Epoch 3/200
24/24 [==============================] - 0s 4ms/step - loss: 138.4796
Epoch 4/200
24/24 [==============================] - 0s 4ms/step - loss: 135.8443
Epoch 5/200
24/24 [==============================] - 0s 4ms/step - loss: 142.3867
Epoch 6/200
24/24 [==============================] - 0s 4ms/step - loss: 135.8671
Epoch 7/200
24/24 [==============================] - 0s 4ms/step - loss: 141.2081
Epoch 8/200
24/24 [==============================] - 0s 4ms/step - loss: 140.5594
Epoch 8: early stopping
Count Down: 7
Epoch 1/200
24/24 [==============================] - 0s 4ms/step - loss: 141.7527
Epoch 2/200
24/24 [===============

In [17]:
metrics_df = pd.DataFrame(columns = ['stock_ID', "RMSE", "MAPE", "MPE", "MTT"])

In [18]:
def mean_positive_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    error = np.mean(np.maximum((y_pred - y_true),0))
    return error

In [19]:
new_rows = []
stock_result = aapl_result
# Calculate RMSE 
rmse = np.sqrt(mean_squared_error(stock_result['ACTUAL'], stock_result['PREDICTED']))
# Calculate MAPE % 
mape = mean_absolute_percentage_error(stock_result['ACTUAL'], stock_result['PREDICTED']) * 100
# Calculate MPE % 
mpe = mean_positive_error(stock_result['ACTUAL'],stock_result['PREDICTED'])
# Calculate MTT in seconds 
mtt = np.mean(stock_result['TRAIN_DURATION'])
new_row = pd.Series(['AAPL',rmse, mape, mpe, mtt], index=metrics_df.columns)
new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)
metrics_df

,stock_ID,RMSE,MAPE,MPE,MTT
0,AAPL,5.372223,2.82082,3.362203,0.980152


In [20]:
new_rows = []
stock_result = f_result
# Calculate RMSE 
rmse = np.sqrt(mean_squared_error(stock_result['ACTUAL'], stock_result['PREDICTED']))
# Calculate MAPE % 
mape = mean_absolute_percentage_error(stock_result['ACTUAL'], stock_result['PREDICTED']) * 100
# Calculate MPE % 
mpe = mean_positive_error(stock_result['ACTUAL'],stock_result['PREDICTED'])
# Calculate MTT in seconds 
mtt = np.mean(stock_result['TRAIN_DURATION'])
new_row = pd.Series(['F',rmse, mape, mpe, mtt], index=metrics_df.columns)
new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)
metrics_df

,stock_ID,RMSE,MAPE,MPE,MTT
0,F,1.131564,6.044729,0.557536,1.124064


In [21]:
new_rows = []
stock_result = nvda_result
# Calculate RMSE 
rmse = np.sqrt(mean_squared_error(stock_result['ACTUAL'], stock_result['PREDICTED']))
# Calculate MAPE % 
mape = mean_absolute_percentage_error(stock_result['ACTUAL'], stock_result['PREDICTED']) * 100
# Calculate MPE % 
mpe = mean_positive_error(stock_result['ACTUAL'],stock_result['PREDICTED'])
# Calculate MTT in seconds 
mtt = np.mean(stock_result['TRAIN_DURATION'])
new_row = pd.Series(['NVDA',rmse, mape, mpe, mtt], index=metrics_df.columns)
new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)
metrics_df

,stock_ID,RMSE,MAPE,MPE,MTT
0,NVDA,11.107127,6.146503,5.909659,1.240483


In [22]:
new_rows = []
stock_result = tsla_result
# Calculate RMSE 
rmse = np.sqrt(mean_squared_error(stock_result['ACTUAL'], stock_result['PREDICTED']))
# Calculate MAPE % 
mape = mean_absolute_percentage_error(stock_result['ACTUAL'], stock_result['PREDICTED']) * 100
# Calculate MPE % 
mpe = mean_positive_error(stock_result['ACTUAL'],stock_result['PREDICTED'])
# Calculate MTT in seconds 
mtt = np.mean(stock_result['TRAIN_DURATION'])
new_row = pd.Series(['TSLA',rmse, mape, mpe, mtt], index=metrics_df.columns)
new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)
metrics_df

,stock_ID,RMSE,MAPE,MPE,MTT
0,TSLA,34.058816,15.604365,25.073614,1.234886


In [23]:
new_rows = []
stock_result = wmt_result
# Calculate RMSE 
rmse = np.sqrt(mean_squared_error(stock_result['ACTUAL'], stock_result['PREDICTED']))
# Calculate MAPE % 
mape = mean_absolute_percentage_error(stock_result['ACTUAL'], stock_result['PREDICTED']) * 100
# Calculate MPE % 
mpe = mean_positive_error(stock_result['ACTUAL'],stock_result['PREDICTED'])
# Calculate MTT in seconds 
mtt = np.mean(stock_result['TRAIN_DURATION'])
new_row = pd.Series(['WMT',rmse, mape, mpe, mtt], index=metrics_df.columns)
new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)
metrics_df

,stock_ID,RMSE,MAPE,MPE,MTT
0,WMT,5.946793,3.32454,1.864607,1.174884
